In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl")
from travel import init_travel
init_travel()

# Load preds

In [2]:
import json

RESULTS_PATH = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_single_debug250/llava-1.5-7b-hf/IterativeVQA_q10_ego4d_single_debug250_llava-1.5-7b-hf_coherence_icl20_20240806224704/outputs_val.json"
METRICS_PATH = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_single_debug250/llava-1.5-7b-hf/IterativeVQA_q10_ego4d_single_debug250_llava-1.5-7b-hf_coherence_icl20_20240806224704/metrics_coherence_val.json"
preds = json.load(open(RESULTS_PATH, "r"))
coherence_metrics = json.load(open(METRICS_PATH, "r"))

In [5]:
low_rel_low_info = []
low_rel_high_info = []
high_rel_low_info = []
high_rel_high_info = []

parallel_idx = 0
for output in preds.values():
    for question, answer in zip(output['questions'], output['answers']):
        relevance = coherence_metrics['metrics_by_example']['relevance'][parallel_idx]
        informativeness = coherence_metrics['metrics_by_example']['informativeness_marginal'][parallel_idx]
        
        if relevance <= 0.2 and informativeness <= 0.2:
            low_rel_low_info.append((question, answer))
        elif relevance <= 0.2 and informativeness >= 0.8:
            low_rel_high_info.append((question, answer))
        elif relevance >= 0.8 and informativeness <= 0.2:
            high_rel_low_info.append((question, answer))
        elif relevance >= 0.8 and informativeness >= 0.8:
            high_rel_high_info.append((question, answer))
        
        parallel_idx += 1
    

IndexError: list index out of range

In [ ]:
from pprint import pprint

pprint(low_rel_high_info)

In [ ]:
from pprint import pprint

pprint(high_rel_low_info)